In [1]:
prompt = "A model that can assess newly submitted menus from kitchens. The model should review the menus based on factors such as dish uniqueness, ingredient availability, pricing, and compliance with regulatory standards. It should also determine the cuisine of each menu using information from the name,description and ingredients. it should highlight any discrepancies or issues for manual review"
temperature = .5
number_of_examples = 5

ANTHROPIC_API_KEY = ""

In [2]:
import os
import random
import requests
import time

def generate_example(prompt, prev_examples, temperature=0.5):
    max_retries = 3
    retry_delay = 1  # seconds
    retry_count = 0

    while retry_count < max_retries:
        try:
            messages = [{"role": "user", "content": f'Now, generate a prompt/response pair for `{prompt}`. Do so in the exact format requested:\n```\n<prompt>prompt</prompt>\n<response>response_goes_here</response>\n```\n\nOnly one prompt/response pair should be generated per turn.'}]

            if len(prev_examples) > 0:
                if len(prev_examples) > 10:
                    prev_examples = random.sample(prev_examples, 10)

                for example in prev_examples:
                    messages.append({
                        "role": "assistant",
                        "content": example
                    })

                    messages.append({
                        "role": "user",
                        "content": 'Now, generate another prompt/response pair. Make it unique.'
                    })

            headers = {
                "x-api-key": ANTHROPIC_API_KEY,
                "anthropic-version": "2023-06-01",
                "content-type": "application/json"
            }

            data = {
                "model": 'claude-3-haiku-20240307', # "claude-3-opus-20240229",
                "max_tokens": 1354,
                "temperature": temperature,
                "messages": messages,
                "system": f"You are generating data which will be used to train a machine learning model.\n\nYou will be given a high-level description of the model we want to train, and from that, you will generate data samples, each with a prompt/response pair.\n\nYou will do so in this format:\n```\n<prompt>prompt</prompt>\n<response>response_goes_here</response>\n```\n\nOnly one prompt/response pair should be generated per turn.\n\nFor each turn, make the example slightly more complex than the last, while ensuring diversity.\n\nMake sure your samples are unique and diverse, yet high-quality and complex enough to train a well-performing model.\n\nHere is the type of model we want to train:\n`{prompt}`"
            }

            response = requests.post("https://api.anthropic.com/v1/messages", headers=headers, json=data)

            response_json = response.json()

            if 'content' in response_json and len(response_json['content']) > 0:
                return '<prompt>' + response_json['content'][0]['text'].split('<prompt>')[1]
            else:
                print("Invalid response format. Retrying...")
                retry_count += 1
                time.sleep(retry_delay)
                
        except Exception as e:
            print(f"An error occurred: {e}. Retrying...")
            retry_count += 1
            time.sleep(retry_delay)

    print("Maximum retries exceeded. Failed to generate example.")
    return None

# Generate examples
prev_examples = []
for i in range(number_of_examples):
    print(f'Generating example {i}')
    example = generate_example(prompt, prev_examples, temperature)
    print(example)
    if example:
        prev_examples.append(example)

print(prev_examples)

Generating example 0
<prompt>Menu for Bella's Bistro:
Appetizers:
- Caprese Salad - Fresh mozzarella, tomatoes, basil, balsamic glaze
- Crispy Calamari - Lightly fried calamari, marinara sauce

Entrees:
- Chicken Parmesan - Breaded chicken breast, marinara, mozzarella, served with spaghetti
- Grilled Salmon - Salmon fillet, lemon-dill sauce, roasted vegetables
- Fettuccine Alfredo - Fettuccine pasta in a creamy Parmesan sauce

Desserts:
- Tiramisu - Espresso-soaked ladyfingers, mascarpone, cocoa
- Cannoli - Crisp pastry shells filled with sweetened ricotta</prompt>
<response>Based on the menu, the cuisine appears to be primarily Italian, with dishes like Caprese salad, Chicken Parmesan, Fettuccine Alfredo, and Italian desserts like tiramisu and cannoli. 

The menu offers a good variety of appetizers, entrees, and desserts, with both meat and seafood options. The pricing seems reasonable for a bistro-style establishment.

The menu complies with standard food safety and labeling regulati

In [3]:
def generate_system_message(prompt):
    headers = {
        "x-api-key": ANTHROPIC_API_KEY,
        "anthropic-version": "2023-06-01",
        "content-type": "application/json"
    }
    data = {
        "model": "claude-3-opus-20240229",
        "max_tokens": 500,
        "temperature": temperature,
        "system":"""
                You will be given a high-level description of the model we are training, and from that, you will generate a simple system prompt for that model to use. Remember, you are not generating the system message for data generation -- you are generating the system message to use for inference. A good format to follow is `Given WHAT_THE_MODEL_SHOULD_DO.`.\n\nMake it as concise as possible. Include nothing but the system prompt in your response.\n\nFor example, never write: `\"SYSTEM_PROMPT_HERE`.\"
                """.strip(),
        "messages": [
            {
                "role": "user",
                "content": f"Here is the prompt: `{prompt.strip()}`. Write a fantastic system message.",
            }
        ],
    }
    response = requests.post("https://api.anthropic.com/v1/messages", headers=headers, json=data)
    return response.json()['content'][0]['text']

system_message = generate_system_message(prompt)

print(f'The system message is: `{system_message}`. Feel free to re-run this cell if you want a better result.')

The system message is: `Given a newly submitted menu from a kitchen, assess it based on dish uniqueness, ingredient availability, pricing, and compliance with regulatory standards. Determine the cuisine of the menu using information from the dish names, descriptions, and ingredients. Highlight any discrepancies or issues found during the assessment for manual review.`. Feel free to re-run this cell if you want a better result.


In [4]:
import pandas as pd

# Initialize lists to store prompts and responses
prompts = []
responses = []

# Parse out prompts and responses from examples
for example in prev_examples:
    try:
        prompt_start = example.index('<prompt>') + len('<prompt>')
        prompt_end = example.index('</prompt>')
        prompt = example[prompt_start:prompt_end].strip()

        response_start = example.index('<response>') + len('<response>')
        response_end = example.index('</response>')
        response = example[response_start:response_end].strip()

        prompts.append(prompt)
        responses.append(response)
    except (ValueError, IndexError):
        pass

# Create a DataFrame
df = pd.DataFrame({
    'prompt': prompts,
    'response': responses
})

# Remove duplicates
df = df.drop_duplicates()

print('There are ' + str(len(df)) + ' successfully-generated examples. Here are the first few:')
df.head()

There are 5 successfully-generated examples. Here are the first few:


,prompt,response
0,Menu for Bella's Bistro:\nAppetizers:\n- Capre...,"Based on the menu, the cuisine appears to be p..."
1,Menu for Jade Garden Chinese Restaurant:\nAppe...,The Jade Garden menu features a variety of aut...
2,Menu for Greenleaf Organic Cafe:\nStarters:\n-...,The Greenleaf Organic Cafe menu showcases a he...
3,Menu for Fusion Flavors Restaurant:\nStarters:...,The Fusion Flavors menu showcases a creative b...
4,Menu for Vegan Delights Cafe:\nStarters:\n- Ja...,The Vegan Delights Cafe menu showcases a diver...


In [5]:
train_df = df.sample(frac=0.9, random_state=42)
test_df = df.drop(train_df.index)

# Save the dataframes to .jsonl files
train_df.to_json('train.jsonl', orient='records', lines=True)
test_df.to_json('test.jsonl', orient='records', lines=True)

In [6]:
from datasets import load_dataset

train_dataset = load_dataset('json', data_files='/kaggle/working/train.jsonl', split="train")
valid_dataset = load_dataset('json', data_files='/kaggle/working/test.jsonl', split="train")

# Preprocess datasets
train_dataset_mapped = train_dataset.map(lambda examples: {'text': [f'[INST] <<SYS>>\n{system_message.strip()}\n<</SYS>>\n\n' + prompt + ' [/INST] ' + response for prompt, response in zip(examples['prompt'], examples['response'])]}, batched=True)
valid_dataset_mapped = valid_dataset.map(lambda examples: {'text': [f'[INST] <<SYS>>\n{system_message.strip()}\n<</SYS>>\n\n' + prompt + ' [/INST] ' + response for prompt, response in zip(examples['prompt'], examples['response'])]}, batched=True)

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [7]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kaggle-environments 1.14.3 requires transformers>=4.33.1, but you have transformers 4.31.0 which is incompatible.


In [8]:
import os
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

2024-04-14 17:06:30.404332: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-14 17:06:30.404449: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-14 17:06:30.517302: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [9]:
# The model that you want to train from the Hugging Face hub
model_name = "NousResearch/Llama-2-7b-chat-hf"

# Fine-tuned model name
new_model = "Llama-2-7b-chat-finetune"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [10]:
# Load dataset (you can process it here)

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset_mapped,
    eval_dataset=valid_dataset_mapped,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

# Train model
trainer.train()

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss


TrainOutput(global_step=1, training_loss=1.5509614944458008, metrics={'train_runtime': 14.8361, 'train_samples_per_second': 0.27, 'train_steps_per_second': 0.067, 'total_flos': 65335590912000.0, 'train_loss': 1.5509614944458008, 'epoch': 1.0})

In [11]:
trainer.model.save_pretrained(new_model)

In [12]:
%load_ext tensorboard
%tensorboard --logdir results/runs

In [13]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "What is a large language model?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will

<s>[INST] What is a large language model? [/INST]  A large language model is a type of artificial intelligence (AI) model that is trained on a large dataset of text to generate language outputs that are coherent and natural-sounding. everybody. These models are typically trained on vast amounts of text data, such as books, articles, and websites, and are designed to learn the patterns and structures of language.

Large language models are often used in natural language processing (NLP) tasks such as language translation, text summarization, and language generation. They are also used in chatbots, virtual assistants, and other applications where language understanding and generation is required.

Some of the key features of large language models include:

1. Deep learning architecture: Large language models are typically built using deep learning architectures such as recurrent neural networks (RNNs), long short-term memory (LSTM) networks,


In [14]:
del model
del pipe
del trainer
import gc
gc.collect()
gc.collect()

0

In [15]:
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [16]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [17]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "Review the following menu and provide an assessment based on dish uniqueness, ingredient availability, pricing, and compliance with regulatory standards. Also determine the cuisine of the menu. Menu: Starters: - Charred Octopus Salad with Citrus Vinaigrette and Micro Greens ($15) - Crispy Pork Belly Bites with Chili-Lime Glaze ($12) Mains: - Miso-Glazed Salmon with Roasted Sweet Potatoes and Sautéed Spinach ($26) - Braised Short Rib Ragu with Pappardelle Pasta and Shaved Parmesan ($28) - Jackfruit 'Carnitas' Tacos with Avocado Crema and Pickled Onions ($20) Desserts: - Coconut Panna Cotta with Mango Compote ($9) - Flourless Chocolate Cake with Raspberry Coulis ($11)"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=1000)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] Review the following menu and provide an assessment based on dish uniqueness, ingredient availability, pricing, and compliance with regulatory standards. Also determine the cuisine of the menu. Menu: Starters: - Charred Octopus Salad with Citrus Vinaigrette and Micro Greens ($15) - Crispy Pork Belly Bites with Chili-Lime Glaze ($12) Mains: - Miso-Glazed Salmon with Roasted Sweet Potatoes and Sautéed Spinach ($26) - Braised Short Rib Ragu with Pappardelle Pasta and Shaved Parmesan ($28) - Jackfruit 'Carnitas' Tacos with Avocado Crema and Pickled Onions ($20) Desserts: - Coconut Panna Cotta with Mango Compote ($9) - Flourless Chocolate Cake with Raspberry Coulis ($11) [/INST]  Based on the menu provided, here is my assessment of the dishes:

Uniqueness:

* Charred Octopus Salad: Unique and interesting combination of flavors and textures, with the charred octopus adding a smoky element to the dish.
* Crispy Pork Belly Bites: A classic dish with a twist, the chili-lime glaze adds